In [ ]:
%%html
<h1 align=center>Caderno de Análise dos Dados do SPED - EFD ICMS - Versão 1 </h1>

In [ ]:
# Importar as bibliotecas para a realização da Análise ((não alterar o código))

from sqlalchemy import create_engine
import pymysql
import datetime as dt
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Conexão ao banco de dados do EFD-Cotribuições (não alterar o código)
conn = pymysql.connect(host="localhost",
                      port=3337,
                      user="spedfiscal",
                      passwd="spedfiscal",
                      database = "master",
                      charset='utf8')

In [ ]:
%%html
<h2 align="center"> Insira o CNPJ da entidade sem espaços, simbolos ou letras </h2>

In [ ]:
cnpj = int(input("Digite o número do CNPJ da entidade: "))

In [ ]:
%%html
<h4 align="center"> Copie na opção abaixo o nome do db para análise </h4>

In [ ]:
# Visualizando os nomes dos Bancos de Dados , entidades e CNPJ
pd.read_sql(f"select cpf_cnpj, nome_contribuinte, datainicial, uf,nomeBD FROM escrituracaofiscal WHERE cpf_cnpj={cnpj}" , con=conn)

In [ ]:
conn.close()

In [ ]:
# Solicita o nome do banco de dados e instancia conn com o novo DB
db_name = input("Digite o nome do Banco de Dados: ")
conn = pymysql.connect(host="localhost",
                      port=3337,
                      user="spedfiscal",
                      passwd="spedfiscal",
                      database = f"{db_name}",
                      charset='utf8')

In [ ]:
%%html
<h2> Análise da Prestação de Serviços de telecomunicação D500 - saída </h2>

In [ ]:
reg_d510_telecom = pd.read_sql("SELECT HASH, COD_ITEM, VL_ITEM, CFOP, ALIQ_ICMS, COD_CTA FROM reg_d510", con=conn)
reg_d510_telecom

In [ ]:
## Soma de todas as Notas Fiscais de saídas do Registro d500:
print(f" Valor total das NFs de saída telecom: R$ {round(reg_d510_telecom.VL_ITEM.sum(),2)}")


In [ ]:
# Soma de valores por código de itens
reg_d510_agg_by_cod_item = reg_d510_telecom[["COD_ITEM","VL_ITEM"]].groupby(["COD_ITEM"]).agg(["sum", "mean","min", "max"])
reg_d510_agg_by_cod_item

In [ ]:
%%html
<h2 align="center"> Soma de todos os itens de telecom </h2>

In [ ]:
# soma de todos das receitas de todos os Serviços de telecom:
total_d510 = reg_d510_agg_by_cod_item["VL_ITEM"]["sum"].sum()
print(f"valor total dos Serviços : R$ {round(total_d510,2)}")

In [ ]:
%%html
<h2 align="center"> Algumas Análises Gráficas </h2>


In [ ]:
plt.figure(figsize=(12,8))
plt.title("Registros d510 - Serviços de telecomunicações por tipo de atividade")
plt.ylabel("Classificação das atividades")
plt.xlabel("Valores dos itens")
sns.barplot(data=reg_d510_telecom, y="COD_ITEM", x="VL_ITEM", hue="COD_CTA")
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.title("Box plot das atividades por serviços de telecomunicações D500")
sns.boxplot(data=reg_d510_telecom, y="COD_ITEM", x="VL_ITEM", hue="COD_CTA")
plt.show()

In [ ]:
conn.close()

In [ ]:
%%html
<h3 align="center"> Execute a célula abaixo Se e somente Se desejar gravar as tabelas no formato excell </h3>

In [ ]:
# Importing files to Excell
month = input("Escreva o nome do mês em minúsculo, sem separações ou símbolos: ")
reg_d510_telecom.to_excel(f"servicos_telecom_d500_{month}.xlsx")
reg_d510_agg_by_cod_item.to_excel(f"soma_agregada_telecom_d500_{month}.xlsx")